# Training model with TTCL layers

This is a pytorch implementation of experiments from [this paper](https://arxiv.org/pdf/1611.03214.pdf)

In [1]:
# !pip install ipywidgets
# !pip install -U tensorly
# !pip install -U tensorly-torch

In [2]:
import tltorch
import torch
from torch import nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import CIFAR10
import torchvision.transforms as T
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from IPython.display import clear_output

C:\Users\Sasha\anaconda3\lib\site-packages\tltorch\factorized_tensors\core.py:145: UserWarning: Creating a subclass of FactorizedTensor TensorizedTensor with no name.
  warnings.warn(f'Creating a subclass of FactorizedTensor {cls.__name__} with no name.')


In [ ]:
from pynvml import *
nvmlInit()
for ind in range(torch.cuda.device_count()):
    print(f'GPU {ind}: ')
    h = nvmlDeviceGetHandleByIndex(ind)
    info = nvmlDeviceGetMemoryInfo(h)
    print(f'\ttotal    : {info.total}')
    print(f'\tfree     : {info.free}')
    print(f'\tused     : {info.used}')

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [4]:
def get_cifar10_transform(train=True):
    if train:
        transform = T.Compose([
            T.RandomCrop(32, padding=4),
            T.RandomHorizontalFlip(),
            T.ToTensor(),
            T.Normalize((0.49139968, 0.48215827 ,0.44653124), (0.24703233, 0.24348505, 0.26158768), inplace=True)

        ])

    else:
        transform = T.Compose([
            T.ToTensor(),
            T.Normalize((0.49139968, 0.48215827 ,0.44653124), (0.24703233, 0.24348505, 0.26158768))
        ])

    return transform

In [5]:
batch_size = 32

train_set = CIFAR10('CIFAR10', train=True, download=True,
                    transform=get_cifar10_transform(train=True))
test_set = CIFAR10('CIFAR10', train=False, download=True,
                   transform=get_cifar10_transform(train=False))
train_loader = DataLoader(train_set, batch_size=batch_size, pin_memory=True, num_workers=2, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, pin_memory=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
def plot_losses_epoch(loss, accuracy, train_losses, test_losses, train_accuracies, test_accuracies, test_top5_accuracies):
    clear_output()
    fig, axs = plt.subplots(2, 2, figsize=(15, 8))
    axs[0][0].plot(range(1, len(train_losses) + 1), train_losses, label='train')
    axs[0][0].plot(range(1, len(test_losses) + 1), test_losses, label='test')
    axs[0][0].set_ylabel('loss')
    axs[0][0].set_xlabel('epoch')
    axs[0][0].legend()

    axs[0][1].plot(range(1, len(train_accuracies) + 1), train_accuracies, label='train')
    axs[0][1].plot(range(1, len(test_accuracies) + 1), test_accuracies, label='test top1')
    axs[0][1].plot(range(1, len(test_top5_accuracies) + 1), test_top5_accuracies, label='test top5')
    axs[0][1].set_ylabel('accuracy')
    axs[0][1].set_xlabel('epoch')
    axs[0][1].legend()

    axs[1][0].plot(range(1, len(loss) + 1), loss)
    axs[1][0].set_ylabel('loss')
    axs[1][0].set_xlabel('batch')

    axs[1][1].plot(range(1, len(accuracy) + 1), accuracy)
    axs[1][1].set_ylabel('accuracy')
    axs[1][1].set_xlabel('batch')

    for r_ax in axs:
        for ax in r_ax:
            ax.grid()

    plt.show()

In [7]:
def save_checkpoint(epoch, model, optimizer, train_loss, test_loss, test_accuracy, train_accuracy, test_top5_accuracy, path):
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'test_loss' : test_loss,
            'test_accuracy' : test_accuracy,
            'train_accuracy' : train_accuracy,
            'test_top5_accuracy': test_top5_accuracy
            }, path)

In [8]:
def train_one_epoch(model, train_dataloader, criterion, optimizer, device="cuda:3", desc='Training...', train_losses=[], test_losses=[], train_accuracies=[], test_accuracies=[], test_top5_accuracies=[]):
    model.to(device)
    model.train()

    losses = []
    accuracies = []
    predicted_classes = torch.Tensor()
    true_classes = torch.Tensor()

    batch_num = 0

    for images, labels in tqdm(train_dataloader, desc=desc):
        optimizer.zero_grad()
        images = images.to(device)
        labels = labels.to(device)
        
        logits = model(images)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        
        losses += [loss.item() * images.shape[0]]
        predicted_classes = torch.cat((predicted_classes, (logits.argmax(dim=-1)).to('cpu')))
        true_classes = torch.cat((true_classes, labels.to('cpu')))
        accuracies += [(logits.argmax(dim=-1).to('cpu') == labels.to('cpu')).double().mean().item()]
        
        if batch_num % 50 == 0:
            plot_losses_epoch(losses, accuracies, train_losses, test_losses, train_accuracies, test_accuracies, test_top5_accuracies)
        batch_num += 1
        
    return losses, predicted_classes.tolist(), true_classes.tolist()


def predict(model, val_dataloader, criterion, device="cuda:3", desc='Evaluating...'):
    model.to(device)
    model.eval()

    val_losses = []
    predicted_classes = torch.Tensor()
    true_classes = torch.Tensor()
    top5_predicted_classes = torch.Tensor()

    for images, labels in tqdm(val_dataloader, desc=desc):

        images = images.to(device)
        labels = labels.to(device)
        
        with torch.no_grad():
            logits = model(images)
            loss = criterion(logits, labels)
        
        val_losses += [loss.item() * images.shape[0]]
        predicted_classes = torch.cat((predicted_classes, (logits.argmax(dim=-1)).to('cpu')))
        true_classes = torch.cat((true_classes, labels.to('cpu')))
        top5_predicted_classes = torch.cat((top5_predicted_classes, (torch.topk(logits, 5).indices.to('cpu')).view(len(images), -1)), dim=0)

    accuracy = (predicted_classes == true_classes).type(torch.DoubleTensor).mean().item()
    top5_accuracy = (top5_predicted_classes == true_classes.view(-1, 1)).any(dim=1).type(torch.DoubleTensor).mean().item()
    return val_losses, accuracy, top5_accuracy

In [9]:
def train(train_losses=[], test_losses=[], train_accuracies=[], test_accuracies=[], test_top5_accuracy=[]):
    train_losses = train_losses
    test_losses = test_losses
    train_accuracies = train_accuracies
    test_accuracies = test_accuracies
    test_top5_accuracies = test_top5_accuracy

    for epoch in range(start_epoch + 1, n_epochs + 1):
        train_losses_epoch, train_predicted_classes, train_true_classes = train_one_epoch(model, train_loader, criterion, optimizer, device, f'Training {epoch}/{n_epochs}', train_losses, test_losses, train_accuracies, test_accuracies, test_top5_accuracies)
        train_losses += [sum(train_losses_epoch) / len(train_losses_epoch)]
        train_accuracies += [(torch.Tensor(train_predicted_classes) == torch.Tensor(train_true_classes)).type(torch.DoubleTensor).mean().item()]
        
        scheduler.step(epoch)

        test_losses_epoch, test_accuracy_estimated, test_top5_accuracy_estimated = predict(model, test_loader, criterion, device, f'Evaluating... {epoch}/{n_epochs}')
        test_losses += [sum(test_losses_epoch) / len(test_losses_epoch)]
        test_accuracies += [test_accuracy_estimated]
        test_top5_accuracies += [test_top5_accuracy_estimated]

        if epoch % save_epoch == 0 and epoch > 0:
            save_checkpoint(epoch=epoch, model=model, optimizer=optimizer, train_loss=train_losses, \
                test_loss=test_losses, test_accuracy=test_accuracies, train_accuracy=train_accuracies, \
                test_top5_accuracy=test_top5_accuracies, path= save_path + f'-epoch{epoch}.pt')
    return train_losses, test_losses, train_accuracies, test_accuracies, test_top5_accuracies

In [10]:
import sys
sys.path.insert(0,'../models')
from TTCL import TTCL
from conv_models import Model, ModelConv

In [11]:
inp_ch, inp_h, inp_w = (3, 32, 32)
p = 0.9

ranks409 = [(20, 20, 20, 1),
            (25, 20, 20, 1),
            (20, 20, 20, 1),
            (20, 20, 20, 1),
            (20, 20, 20, 1)]

ranks325 = [(20, 20, 20, 1),
            (27, 22, 22, 1),
            (23, 23, 23, 1),
            (23, 23, 23, 1),
            (23, 23, 23, 1)]

ranks233 = [(25, 25, 25, 1),
            (30, 27, 27, 1),
            (27, 27, 27, 1),
            (27, 27, 27, 1),
            (27, 27, 27, 1)]

ranks206 = [(27, 27, 27, 1),
            (30, 30, 27, 1),
            (30, 30, 27, 1),
            (30, 30, 27, 1),
            (30, 30, 27, 1)]

ranks = ranks409

In [ ]:
baseline_number_of_parameters = 557642

model = Model(ranks=ranks, p=p, device=device)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'Number of parameters: {count_parameters(model)}')
print(f'Compression ratio: {(baseline_number_of_parameters / count_parameters(model)):.2f}')

compression_ratio=baseline_number_of_parameters / count_parameters(model)

In [ ]:
from torch.optim.lr_scheduler import MultiStepLR
start_epoch = 60
RESUME = False
criterion = nn.CrossEntropyLoss()

n_epochs = 100
save_epoch = 10

inp_ch, inp_h, inp_w = (3, 32, 32)

stats = []
model = Model(ranks=ranks, p=p, device=device)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1, momentum=0.9)
train_losses, test_losses, train_acc, test_acc, test_top5_acc = [], [], [], [], []
start_epoch = 0

import os
dir_path = f'trained_models/'
os.makedirs(dir_path, exist_ok=True)
save_path = dir_path + f'TTCL-p-{p}-comp-{compression_ratio:.2f}'

milestones = [30, 60, 90]
scheduler = MultiStepLR(optimizer, milestones=milestones, gamma=0.1)
cur_train_losses, cur_test_losses, cur_train_accuracies, cur_test_accuracies, cur_test_top5_accuracies = train(train_losses, test_losses, train_acc, test_acc, test_top5_acc)
stats.append({'train_losses' : cur_train_losses,
            'test_losses' : cur_test_losses,
            'train_accuracies' : cur_train_accuracies,
            'test_accuracies' : cur_test_accuracies,
            'test_top5_accuracies' : cur_test_top5_accuracies})
RESUME = False